### Amazon Sentiment Data

In [ ]:
import lxmls.readers.sentiment_reader as srs
from lxmls.deep_learning.utils import Data
corpus = srs.SentimentCorpus("books")
data = Data(corpus=corpus)

### Exercise 2.2 Implement Backpropagation for an MLP in Numpy and train it

In [ ]:
# Model
geometry = [corpus.nr_features, 20, 2]
activation_functions = ['sigmoid', 'softmax']

# Optimization
learning_rate = 0.05
num_epochs = 10
batch_size = 30

In [ ]:
from lxmls.deep_learning.numpy_models.mlp import NumpyMLP
model = NumpyMLP(
    geometry=geometry,
    activation_functions=activation_functions,
    learning_rate=learning_rate
)

#### Milestone 1:
Check gradients using the empirical gradient computation

In [ ]:
from lxmls.deep_learning.numpy_models.mlp import cross_entropy_loss, backpropagation
batch = data.batches('train', batch_size=batch_size)[0]

In [ ]:
# Select one parameter from the network
layer_index = 0
is_bias = 0
row = 1
column = batch['input'].nonzero()[1][2]

In [ ]:
import numpy as np
from copy import deepcopy

def perturbated_parameters(parameters, perturbation_range, layer_index=None, is_bias=None, row=None, column=None):
    
    # Deep copy to avoid changing parameters 
    parameters = deepcopy(parameters)
    
    # Select the weight and get range
    if is_bias:
        # bias
        study_weight = float(parameters[layer_index][is_bias][row])
    else:
        # weight
        study_weight = float(parameters[layer_index][is_bias][row, column])
    for perturbation in perturbation_range:
        if is_bias:
            parameters[layer_index][is_bias][row] = study_weight + perturbation
        else:
            parameters[layer_index][is_bias][row, column] = study_weight + perturbation
        yield parameters

In [ ]:
study_weight = model.parameters[layer_index][is_bias][row, column]
study_loss = cross_entropy_loss(batch['input'], batch['output'], model.parameters)

In [ ]:
perturbation = np.linspace(-6, 6, 200)
loss_range = []
for parameters in perturbated_parameters(
    model.parameters,
    perturbation,
    layer_index=layer_index,
    is_bias=is_bias,
    row=row,
    column=column
):
    loss_range.append(cross_entropy_loss(batch['input'], batch['output'], parameters))
loss_range = np.array(loss_range)
weight_range = perturbation + study_weight

In [ ]:
gradient = backpropagation(batch['input'], batch['output'], model.parameters)
weight_gradient = -gradient[layer_index][is_bias][row, column]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
# Plot empirical
plt.plot(weight_range, loss_range)
plt.plot(study_weight, study_loss, 'xr')
# Plot real
h = plt.plot(weight_range, weight_gradient*(weight_range - study_weight) + study_loss, 'r--')

#### Milestone 2:
Train a MLP

In [ ]:
# Get batch iterators for train and test
train_batches = data.batches('train', batch_size=batch_size)
test_set = data.batches('test', batch_size=None)[0]

# Epoch loop
for epoch in range(num_epochs):

    # Batch loop
    for batch in train_batches:
        model.update(input=batch['input'], output=batch['output'])

    # Prediction for this epoch
    hat_y = model.predict(input=test_set['input'])

    # Evaluation
    accuracy = 100*np.mean(hat_y == test_set['output'])

    # Inform user
    print("Epoch %d: accuracy %2.2f %%" % (epoch+1, accuracy))